In [2]:
!nvidia-smi

Sat Sep 13 14:56:20 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   43C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip install transformers[sentencepiece] datasets sacrebleu rouge_score py7zr -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.1/104.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.7/69.7 kB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.0/97.0 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.7/51.7 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.7/142.7 kB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.6/413.6 kB 36.1 MB/s eta 0:00:00


In [ ]:
!pip install --upgrade accelerate
!pip uninstall -y transformers accelerate
!pip install transformers accelerate

Found existing installation: transformers 4.56.1
Uninstalling transformers-4.56.1:
  Successfully uninstalled transformers-4.56.1
Found existing installation: accelerate 1.10.1
Uninstalling accelerate-1.10.1:
  Successfully uninstalled accelerate-1.10.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 124.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.9/374.9 kB 35.8 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline, set_seed, AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk
from evaluate import load
import matplotlib.pyplot as plt
import pandas as pd
import torch
from tqdm import tqdm
import nltk
from nltk.tokenize import sent_tokenize
nltk.download("punkt")

In [4]:
pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.6 MB/s eta 0:00:00


In [6]:
device = 'cuda' if torch.cuda.is_available() else "cpu"
device

'cuda'

In [7]:
modelCKPT = "google/pegasus-cnn_dailymail"
tokenizer = AutoTokenizer.from_pretrained(modelCKPT)
modelPegasus = AutoModelForSeq2SeqLM.from_pretrained(modelCKPT).to(device)

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/88.0 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

spiece.model:   0%|          | 0.00/1.91M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/65.0 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.28G [00:00<?, ?B/s]

Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-cnn_dailymail and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


generation_config.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

In [ ]:
# download and unzip data

!wget https://github.com/basajjabaka/textSummarizer/raw/refs/heads/main/summarizer-data.zip
!unzip summarizer-data.zip

--2025-09-13 14:59:21--  https://github.com/entbappy/Branching-tutorial/raw/master/summarizer-data.zip
Resolving github.com (github.com)... 140.82.116.3
Connecting to github.com (github.com)|140.82.116.3|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip [following]
--2025-09-13 14:59:22--  https://raw.githubusercontent.com/entbappy/Branching-tutorial/master/summarizer-data.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7903594 (7.5M) [application/zip]
Saving to: ‘summarizer-data.zip’

summarizer-data.zip 100%[===================>]   7.54M  --.-KB/s    in 0.1s    

2025-09-13 14:59:22 (57.7 MB/s) - ‘summarizer-data.zip’ saved [7903594/790

In [9]:
samsumDF = load_from_disk('samsum_dataset')
samsumDF

DatasetDict({
    train: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 14732
    })
    test: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 819
    })
    validation: Dataset({
        features: ['id', 'dialogue', 'summary'],
        num_rows: 818
    })
})

In [10]:
splitLens = [len(samsumDF[split]) for split in samsumDF]
print(f"Split lengths: {splitLens}")
print(f"Features: {samsumDF['train'].column_names}")
print("\nDialogue:")

print(samsumDF['test'][1]['dialogue'])

print("\nSummary:")
print(samsumDF['test'][1]['summary'])

Split lengths: [14732, 819, 818]
Features: ['id', 'dialogue', 'summary']

Dialogue:
Eric: MACHINE!
Rob: That's so gr8!
Eric: I know! And shows how Americans see Russian ;)
Rob: And it's really funny!
Eric: I know! I especially like the train part!
Rob: Hahaha! No one talks to the machine like that!
Eric: Is this his only stand-up?
Rob: Idk. I'll check.
Eric: Sure.
Rob: Turns out no! There are some of his stand-ups on youtube.
Eric: Gr8! I'll watch them now!
Rob: Me too!
Eric: MACHINE!
Rob: MACHINE!
Eric: TTYL?
Rob: Sure :)

Summary:
Eric and Rob are going to watch a stand-up on youtube.


In [11]:
def examples2features(exampleBatch):
  inputEncodings = tokenizer(exampleBatch['dialogue'], max_length=1024, truncation=True)

  with tokenizer.as_target_tokenizer():
    targetEncodings = tokenizer(exampleBatch['summary'], max_length=128, truncation=True)

  return{
      'input_ids': inputEncodings['input_ids'],
      'attention_mask': inputEncodings['attention_mask'],
      'labels': targetEncodings['input_ids']
  }


In [12]:
samsumDF_pt = samsumDF.map(examples2features, batched=True)

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:4007: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

In [13]:
samsumDF_pt['train']

Dataset({
    features: ['id', 'dialogue', 'summary', 'input_ids', 'attention_mask', 'labels'],
    num_rows: 14732
})

In [14]:
# Training

from transformers import DataCollatorForSeq2Seq, TrainingArguments, Trainer

seq2seqDataCollator = DataCollatorForSeq2Seq(tokenizer, model=modelPegasus)

trainerArgs = TrainingArguments(
    output_dir='pegasus-samsum',
    num_train_epochs=1,
    warmup_steps=500,
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    weight_decay=0.01,
    logging_steps=10,
    eval_strategy='steps',
    eval_steps=500,
    save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(
    model=modelPegasus,
    args=trainerArgs,
    tokenizer=tokenizer,
    data_collator=seq2seqDataCollator,
    train_dataset=samsumDF_pt['train'], # always use "train" in this scenario, "test" has been used because it is not large
    eval_dataset=samsumDF_pt['validation']
)

/tmp/ipython-input-3740871810.py:21: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [15]:
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'bos_token_id': None}.
/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: basajjabaka (basajjabaka-uganda-bureau-of-statistics) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss


/usr/local/lib/python3.12/dist-packages/transformers/modeling_utils.py:4034: UserWarning: Moving the following attributes in the config to the generation config: {'max_length': 128, 'min_length': 32, 'num_beams': 8, 'length_penalty': 0.8}. You are seeing this warning because you've set generation parameters in the model config, as opposed to in the generation config.
  warnings.warn(


TrainOutput(global_step=52, training_loss=2.978932233957144, metrics={'train_runtime': 281.4033, 'train_samples_per_second': 2.91, 'train_steps_per_second': 0.185, 'total_flos': 314017624350720.0, 'train_loss': 2.978932233957144, 'epoch': 1.0})

In [16]:
# Evaluation

def generate_batch_sized_chunks(elementsList, batchSize):
  """split the dataset into smaller batches that we can process simultaneously
  Yield successive batch-sized chunks from list"""

  for i in range(0, len(elementsList), batchSize):
    yield elementsList[i:i+batchSize]

def calculate_metric_on_test_ds(dataset, metric, model,
                                tokenizer, batchSize=16, device=device,
                                columnText='article', columnSummary='highlights'):
  articleBatches = list(generate_batch_sized_chunks(dataset[columnText], batchSize))
  targetBatches = list(generate_batch_sized_chunks(dataset[columnSummary], batchSize))

  for articleBatch, targetBatch in tqdm(zip(articleBatches, targetBatches), total=len(articleBatches)):

    inputs = tokenizer(articleBatch, max_length=1024, truncation=True,
                       padding='max_length', return_tensors='pt')
    summaries = model.generate(input_ids=inputs['input_ids'].to(device),
                               attention_mask=inputs['attention_mask'].to(device),
                               length_penalty=0.8, num_beams=8, max_length=128)
    """parameter for length penalty ensures the model doesnt generate sequences that are too long"""

    # Finally, we decode the generated texts
    # replace the token, and add the decoded texts with the reference to the metris
    decodedSummaries = [tokenizer.decode(s, skip_special_tokens=True,
                                        clean_up_tokenization_spaces=True) for s in summaries]
    decodedSummaries = [d.replace("", " ") for d in decodedSummaries]

    metric.add_batch(predictions=decodedSummaries, references=targetBatch)

  #  Finally compute and return the ROUGE scores
  score = metric.compute()
  return score

In [18]:
rougeNames = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
rougeMetric = load('rouge')

In [17]:
pip install rouge_score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24934 sha256=c0afbbf74b5b1196e7e91d4b4f24c5d36212e63a74bd38cb27e22a3c778daede
  Stored in directory: /root/.cache/pip/wheels/85/9d/af/01feefbe7d55ef5468796f0c68225b6788e85d9d0a281e7a70
Successfully built rouge_score


In [20]:
score = calculate_metric_on_test_ds(
    samsumDF['test'], # here use the whole test data not just the first 10 rows
    rougeMetric,
    modelPegasus,
    tokenizer,
    batchSize=2,
    columnText='dialogue',
    columnSummary='summary')

rougeDict = dict((rn, score[rn]) for rn in rougeNames)
pd.DataFrame(data=rougeDict, index=[f'pegasus'])

100%|██████████| 410/410 [37:18<00:00,  5.46s/it]


,rouge1,rouge2,rougeL,rougeLsum
pegasus,0.016372,0.000233,0.016223,0.016273


In [21]:
#save model

modelPegasus.save_pretrained("pegasus-samsum-model")

In [22]:
#save tokenizer

tokenizer.save_pretrained("TextSummarizertokenizer")

('TextSummarizertokenizer/tokenizer_config.json',
 'TextSummarizertokenizer/special_tokens_map.json',
 'TextSummarizertokenizer/spiece.model',
 'TextSummarizertokenizer/added_tokens.json',
 'TextSummarizertokenizer/tokenizer.json')

In [23]:
# load tokenizer

tokenizer = AutoTokenizer.from_pretrained("/content/TextSummarizertokenizer")

In [24]:
# Prediction

genArgs = {
    "length_penalty": 0.8,
    "num_beams": 8,
    "max_length": 128
}

sampleText = samsumDF['test'][0]['dialogue']
reference = samsumDF['test'][0]['summary']

genText = pipeline("summarization", model="pegasus-samsum-model", tokenizer=tokenizer)

print("Dialogue:")
print(sampleText)

print("\nReference Summary:")
print(reference)

print("\nModel Summary:")
print(genText(sampleText, **genArgs)[0]['summary_text'])

Device set to use cuda:0
Your max_length is set to 128, but your input_length is only 122. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=61)


Dialogue:
Hannah: Hey, do you have Betty's number?
Amanda: Lemme check
Hannah: <file_gif>
Amanda: Sorry, can't find it.
Amanda: Ask Larry
Amanda: He called her last time we were at the park together
Hannah: I don't know him well
Hannah: <file_gif>
Amanda: Don't be shy, he's very nice
Hannah: If you say so..
Hannah: I'd rather you texted him
Amanda: Just text him 🙂
Hannah: Urgh.. Alright
Hannah: Bye
Amanda: Bye bye

Reference Summary:
Hannah needs Betty's number but Amanda doesn't have it. She needs to contact Larry.

Model Summary:
Amanda: Ask Larry Amanda: He called her last time we were at the park together .<n>Hannah: I'd rather you texted him .<n>Amanda: Just text him .
